In [90]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [91]:
df = pd.read_csv("../data/bangkok_traffy.csv")

In [92]:
df = df[["type", "comment", "province"]]
df = df[df['province'].str.contains('กรุงเทพ|bangkok',na=False,case=False)]
df = df.drop(["province"], axis=1)

df['type'].dropna(axis=0, inplace=True)
df['comment'].dropna(axis=0, inplace=True)

df['type'].fillna('', inplace=True) 
df['type'] = df['type'].astype(str) 
df['type'] = df['type'].str.strip('{}').apply(lambda x :[e for e in x.split(',') if e])
df = df.dropna().reset_index(drop=True)

In [93]:
df.isna().sum()

type       0
comment    0
dtype: int64

In [94]:
# import re
# import string
# import pythainlp
# from tqdm.auto import tqdm
# tqdm.pandas()

# def preprocessing_comment(comment):
#     # Normalize text
#     try:
#       comment = pythainlp.util.normalize(comment)
#     except:
#       print(comment)

#     # Tokenize text
#     tokens = pythainlp.word_tokenize(comment)
  
#     return comment

# df['comment_clean'] = df['comment'].progress_apply(preprocessing_comment)

In [95]:
df.head()

,type,comment
0,[],ไม่มีภาพ
1,[ความสะอาด],ขยะเยอะ
2,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...


In [96]:
df["type"] = df["type"].apply(lambda x: np.nan if x == [] else x)
df = df.dropna().reset_index(drop=True)
df.head()

,type,comment
0,[ความสะอาด],ขยะเยอะ
1,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
2,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
3,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
4,"[ถนน, ร้องเรียน, น้ำท่วม, ความปลอดภัย]",1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...


In [97]:
df.isna().sum()

type       0
comment    0
dtype: int64

In [98]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
transformed_data = mlb.fit_transform(df['type'])

dummy_df = pd.DataFrame(transformed_data, columns=[f'{label}' for label in mlb.classes_]).reset_index(drop=True)

In [99]:
# df.type

In [100]:
dummy_df.sum(axis=0).sort_values(ascending=False)

ถนน            77609
ทางเท้า        31816
แสงสว่าง       22885
ความปลอดภัย    20869
ความสะอาด      20139
น้ำท่วม        19197
กีดขวาง        15699
ท่อระบายน้ำ    14129
สะพาน          12276
จราจร          12252
สายไฟ           9720
เสียงรบกวน      8899
คลอง            8762
ต้นไม้          8171
ร้องเรียน       7717
ป้าย            6609
สัตว์จรจัด      4251
PM2.5           1180
สอบถาม          1171
เสนอแนะ         1002
คนจรจัด          991
การเดินทาง       548
ห้องน้ำ          276
ป้ายจราจร        257
dtype: int64

In [101]:
def apply_1(x):
        if (len(x) == 1) and (["ร้องเรียน"] in x):
            x = ["other"]
        else:
            x = [elem for elem in x if elem not in ["ร้องเรียน"]]
        return x
def preprocess_type(df):
    df = df.copy()
    other_list = ["PM2.5", "สอบถาม", "เสนอแนะ", "คนจรจัด", "การเดินทาง", "ห้องน้ำ", "ป้ายจราจร"]
    df['type'] = df['type'].progress_apply(lambda x: [elem if elem not in other_list else 'other' for elem in x])
    df["type"] = df["type"].apply(lambda x: ["other"] if x == [] else x)
    df['type'] = df['type'].progress_apply(apply_1)
    df["type"] = df["type"].apply(lambda x: ["other"] if x == [] else x)
    return df

tmp_df = preprocess_type(df)

100%|██████████| 208999/208999 [00:00<00:00, 390244.98it/s]


In [102]:
tmp_df["type"] = tmp_df["type"].apply(lambda x: np.nan if x == [] else x)
tmp_df.isna().sum()

type       0
comment    0
dtype: int64

In [103]:
# tmp = "ร้องเรียน"
# filtered_df = tmp_df[tmp_df['type'].progress_apply(lambda x: any(tmp in elem for elem in x))]
# filtered_df.type.tolist()

In [104]:
tmp_df.sample(20)

,type,comment
6771,"[คลอง, ความสะอาด]",แจ้งปัญหาน้ำเน่าเสียและขยะ น้ำมีกลิ่นเหม็นเสี่...
171635,[other],สวนย่อมอยู่ในซอยพรานนก 7 แยกสุดสาคร 13 สถานที่...
83526,[น้ำท่วม],น้ำท่วม
5168,"[ถนน, สายไฟ]",1. ถนนเส้นรามคำแหง ทำถนน รถไฟฟ้า สายไฟ แย่มาก
99332,"[ถนน, แสงสว่าง]",ไฟส่องสว่างถนนหลวงแพ่ง แขวงขุมทอง เขตลาดกระบัง...
189191,"[ถนน, ทางเท้า, ความปลอดภัย]",ปัญหา : บาทวิถี เนื่องจาก ฟุตบาทแตกชำรุด พื้นท...
106851,"[ถนน, ความปลอดภัย]",ถนนเป็นหลุม ขอบท่อชัดเจน อันตรายมากโดยเฉพาะคนข...
184653,[กีดขวาง],ทำไมงานวิ่งสวนเบญจกิติ ต้องมาใช้ทางเข้าสวนเป็น...
77325,[ความปลอดภัย],สายสืาอสารพันยุ่งเหยิงมากครับ กลัวจะเกิดอันตรา...
170623,[ถนน],‘กรุณาอ่านข้อมูลบรรยายประกอบการปักหมุด’\n* ปัญ...


In [105]:
transformed_data = mlb.fit_transform(tmp_df['type'])

dummy_df = pd.DataFrame(transformed_data, columns=[f'type_{label}' for label in mlb.classes_]).reset_index(drop=True)
dummy_df.sum(axis=0).sort_values(ascending=False)

type_ถนน            77609
type_ทางเท้า        31816
type_แสงสว่าง       22885
type_ความปลอดภัย    20869
type_ความสะอาด      20139
type_น้ำท่วม        19197
type_กีดขวาง        15699
type_ท่อระบายน้ำ    14129
type_สะพาน          12276
type_จราจร          12252
type_สายไฟ           9720
type_เสียงรบกวน      8899
type_คลอง            8762
type_ต้นไม้          8171
type_ป้าย            6609
type_other           6558
type_สัตว์จรจัด      4251
dtype: int64

In [106]:
# dummy_df.sum(axis=0).sort_values(ascending=True).index.tolist()

In [108]:
def apply_filter_type(x):
    order_list = ['สัตว์จรจัด',
 'other',
 'ป้าย',
 'ต้นไม้',
 'คลอง',
 'เสียงรบกวน',
 'สายไฟ',
 'จราจร',
 'สะพาน',
 'ท่อระบายน้ำ',
 'กีดขวาง',
 'น้ำท่วม',
 'ความสะอาด',
 'ความปลอดภัย',
 'แสงสว่าง',
 'ทางเท้า',
 'ถนน']
    for order in order_list:
        if order in x:
            return order
    print(x)
        
tmp_df_2 = tmp_df.copy()
tmp_df_2 = tmp_df_2["type"].apply(apply_filter_type)
tmp_df_2

0           ความสะอาด
1               สายไฟ
2               สะพาน
3             น้ำท่วม
4             น้ำท่วม
             ...     
208994    ท่อระบายน้ำ
208995      ความสะอาด
208996           คลอง
208997            ถนน
208998            ถนน
Name: type, Length: 208999, dtype: object